In [1]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
# Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

# Declaring the list of empty variables, So that we can append the data overall
sr = []
movie_name = []
year = []
time=[]
genre = []
director = []
rating=[]
cast = []
synopsis = []
metascore =[]
votes = []

pages = np.arange(1,133350,50)

In [7]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=tv_movie&start="+str(page)+"&view=advanced")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item-content'})
    sleep(randint(2,8))
    for store in movie_data:

        #  serial number
        sr.append(page)

        # name
        name = store.h3.a.text
        movie_name.append(name)
        
        # releasing year
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace(" TV Movie", "").replace("(", "").replace(")", "") if store.h3.find('span', class_ = "lister-item-year text-muted unbold") else "Not defined"
        year.append(year_of_release)
        
        # runtime
        runtime = store.p.find("span", class_ = 'runtime').text if store.find('span', class_ = "runtime") else "Not defined"
        time.append(runtime)
        
        # rating
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else "Not yet rated"
        rating.append(rate)

        # synopsis
        syno = store.find_all('p', class_ = 'text-muted')
        syno_ = syno[1].text.replace('\n', '') if len(syno) > 1 else 'No synopsis'
        synopsis.append(syno_)

        # Director 
        direct = store.find_all('a', class_ = '') 
        direct_ = direct[12].text if len(direct) > 12 else 'Unknown'
        director.append(direct_)

        # cast
        act = store.find_all('a', class_ = '') 
        actors_ = []
        for i in range (13, len(act)):
            actors_.append(act[i].text if len(act) > 13 else 'Unknown')
        cast.append(actors_)

        # genre
        gen = store.p.find("span", class_ = 'genre').text.replace("            ", "").replace("\n", "") if store.p.find('span', class_ = "genre") else "Not given"
        genre.append(gen)
        
        # votes
        value = store.find_all('span', attrs = {'name': "nv"})
        vote = value[0].text if len(value) > 0 else 'Not voted'
        votes.append(vote)
        

In [5]:

movies = pd.DataFrame({"Title": movie_name, "Releasing year" : year, "Watch Time": time,"Rating": rating, "Genre": genre, "Votes": votes,  "Synopsis": synopsis, "Cast": cast, "Director": director})

In [6]:
#movies.to_excel("TV shows.xlsx")
movies.to_csv("TV shows.csv", mode = "a")
